In [1]:
import csv
import os
import nltk
import pickle

In [2]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [3]:
file_name = "lyrics_1989_onwards.csv"

# Read the csv file
with open(file_name, 'r') as f:
    reader = csv.reader(f)
    data = list(reader)


In [4]:
# convert ot pandas
import pandas as pd
df = pd.DataFrame(data)
df.columns = ['song', 'artist', 'lyrics']


In [5]:
# remove first row
df = df.iloc[1:]

In [6]:
df

,song,artist,lyrics
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...
...,...,...,...
757,Emotions,Mariah Carey,Emotions Lyrics[Chorus]\nYou've got me feeling...
758,Good Vibrations,Marky Mark & The Funky Bunch Featuring Loleatt...,
759,I Adore Mi Amor,Color Me Badd,"I Adore Mi Amor LyricsDream on, dream away\nI ..."
760,I Don't Wanna Cry,Mariah Carey,I Don’t Wanna Cry Lyrics[Verse 1]\nOnce again ...


In [7]:
# tokenize the lyrics
from nltk.tokenize import RegexpTokenizer

In [8]:
# tokenise
tokenizer = RegexpTokenizer(r'\w+')

df['tokens'] = df['lyrics'].apply(tokenizer.tokenize)

In [9]:
df

,song,artist,lyrics,tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[Another, Day, in, Paradise, Lyrics, Verse, 1,..."
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[Baby, Don, t, Forget, My, Number, LyricsOkay,..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[My, Rap, CD, Collection, LyricsA, Hempstead, ..."
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[Blame, It, on, the, Rain, Lyrics, Verse, 1, Y..."
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[Cold, Hearted, Lyrics, Chorus, He, s, a, cold..."
...,...,...,...,...
757,Emotions,Mariah Carey,Emotions Lyrics[Chorus]\nYou've got me feeling...,"[Emotions, Lyrics, Chorus, You, ve, got, me, f..."
758,Good Vibrations,Marky Mark & The Funky Bunch Featuring Loleatt...,,[]
759,I Adore Mi Amor,Color Me Badd,"I Adore Mi Amor LyricsDream on, dream away\nI ...","[I, Adore, Mi, Amor, LyricsDream, on, dream, a..."
760,I Don't Wanna Cry,Mariah Carey,I Don’t Wanna Cry Lyrics[Verse 1]\nOnce again ...,"[I, Don, t, Wanna, Cry, Lyrics, Verse, 1, Once..."


In [11]:
# remove numeric, stop words and short words
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string

stop_words = set(stopwords.words('english'))

# add pronouns to stop words

# list of pronouns

song_specific =  ['lyrics', 'verse']

pronouns = ['the', 'i', 'me', 'my', 'mine', 'myself', 'we', 'us', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']

pronouns = pronouns + song_specific

pronouns = set(pronouns)

stop_words = stop_words.union(pronouns)

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# remove profanities?

# https://www.freewebheaders.com/youtube-blacklist-words-free-and-youtube-comment-moderation/

with open('profanity.txt', 'r') as file:
    # split by comma
    profanities = file.read().split(',')
    profanities = [w.strip() for w in profanities]

def remove_stopwords(tokens):
    return [w for w in tokens if w not in stop_words]

def remove_punctuation(tokens):
    return [w for w in tokens if not w in string.punctuation]

def remove_numeric(tokens):
    return [w for w in tokens if not w.isnumeric()]

def remove_short_words(tokens):
    return [w for w in tokens if len(w) > 2]

def lemmatize(tokens):
    return [lemmatizer.lemmatize(w) for w in tokens]

def remove_profanity(tokens):
    return [w for w in tokens if not w in profanities]


In [12]:
def preprocess_pipeline(tokens):
    # lower case
    tokens = [w.lower() for w in tokens]
    tokens = remove_stopwords(tokens)
    tokens = remove_punctuation(tokens)
    tokens = remove_numeric(tokens)
    tokens = remove_short_words(tokens)
    tokens = lemmatize(tokens)

    # remove profanity
    tokens = remove_profanity(tokens)
    
    return tokens

In [13]:
df["processed_tokens"]  = df["tokens"].apply(preprocess_pipeline)

In [14]:
df

,song,artist,lyrics,tokens,processed_tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[Another, Day, in, Paradise, Lyrics, Verse, 1,...","[another, day, paradise, call, man, street, si..."
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[Baby, Don, t, Forget, My, Number, LyricsOkay,...","[baby, forget, number, lyricsokay, okay, eddie..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[My, Rap, CD, Collection, LyricsA, Hempstead, ...","[rap, collection, lyricsa, hempstead, high, lo..."
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[Blame, It, on, the, Rain, Lyrics, Verse, 1, Y...","[blame, rain, said, need, told, goodbye, goodb..."
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[Cold, Hearted, Lyrics, Chorus, He, s, a, cold...","[cold, hearted, chorus, cold, hearted, snake, ..."
...,...,...,...,...,...
757,Emotions,Mariah Carey,Emotions Lyrics[Chorus]\nYou've got me feeling...,"[Emotions, Lyrics, Chorus, You, ve, got, me, f...","[emotion, chorus, got, feeling, emotion, deepe..."
758,Good Vibrations,Marky Mark & The Funky Bunch Featuring Loleatt...,,[],[]
759,I Adore Mi Amor,Color Me Badd,"I Adore Mi Amor LyricsDream on, dream away\nI ...","[I, Adore, Mi, Amor, LyricsDream, on, dream, a...","[adore, amor, lyricsdream, dream, away, think,..."
760,I Don't Wanna Cry,Mariah Carey,I Don’t Wanna Cry Lyrics[Verse 1]\nOnce again ...,"[I, Don, t, Wanna, Cry, Lyrics, Verse, 1, Once...","[wanna, cry, sit, silence, said, done, emptine..."


In [15]:
# remove any song with 0 tokens
df = df[df['processed_tokens'].map(len) > 0]

In [21]:
# create list of all tokens 
all_tokens = df['processed_tokens'].tolist()

In [22]:
from gensim.corpora import Dictionary
dictionary = Dictionary(all_tokens)
dictionary.filter_extremes(no_below = 20, no_above = 0.8)

In [23]:
from gensim.corpora import MmCorpus
gensim_corpus = [dictionary.doc2bow(song) for song in all_tokens]
temp = dictionary[0]
id2word = dictionary.id2token

In [34]:
chunksize = 2000
passes = 20
iterations = 400
num_topics = 4

In [35]:
from gensim.models import LdaModel
lda_model = LdaModel(
corpus=gensim_corpus,
id2word=id2word,
chunksize=chunksize,
alpha='auto',
eta='auto',
iterations=iterations,
num_topics=num_topics,
passes=passes
)

In [36]:
from gensim.models.coherencemodel import CoherenceModel
coherencemodel = CoherenceModel(model=lda_model, texts=all_tokens, dictionary=dictionary, coherence='c_v')
print(coherencemodel.get_coherence())

0.4706439045829762


In [37]:
df

,song,artist,lyrics,tokens,processed_tokens
1,Another Day In Paradise,Phil Collins,Another Day in Paradise Lyrics[Verse 1]\nShe c...,"[Another, Day, in, Paradise, Lyrics, Verse, 1,...","[another, day, paradise, call, man, street, si..."
2,Baby Don't Forget My Number,Milli Vanilli,"Baby Don’t Forget My Number LyricsOkay\nOkay, ...","[Baby, Don, t, Forget, My, Number, LyricsOkay,...","[baby, forget, number, lyricsokay, okay, eddie..."
3,"Batdance (From ""Batman"")",Prince,My Rap CD Collection LyricsA\n\n- Hempstead Hi...,"[My, Rap, CD, Collection, LyricsA, Hempstead, ...","[rap, collection, lyricsa, hempstead, high, lo..."
4,Blame It On The Rain,Milli Vanilli,Blame It on the Rain Lyrics[Verse 1]\nYou said...,"[Blame, It, on, the, Rain, Lyrics, Verse, 1, Y...","[blame, rain, said, need, told, goodbye, goodb..."
5,Cold Hearted,Paula Abdul,Cold Hearted Lyrics[Chorus]\nHe's a cold-heart...,"[Cold, Hearted, Lyrics, Chorus, He, s, a, cold...","[cold, hearted, chorus, cold, hearted, snake, ..."
...,...,...,...,...,...
755,Coming Out Of The Dark,Gloria Estefan,Coming Out of the Dark Lyrics[Verse 1]\nWhy be...,"[Coming, Out, of, the, Dark, Lyrics, Verse, 1,...","[coming, dark, afraid, alone, though, life, ne..."
757,Emotions,Mariah Carey,Emotions Lyrics[Chorus]\nYou've got me feeling...,"[Emotions, Lyrics, Chorus, You, ve, got, me, f...","[emotion, chorus, got, feeling, emotion, deepe..."
759,I Adore Mi Amor,Color Me Badd,"I Adore Mi Amor LyricsDream on, dream away\nI ...","[I, Adore, Mi, Amor, LyricsDream, on, dream, a...","[adore, amor, lyricsdream, dream, away, think,..."
760,I Don't Wanna Cry,Mariah Carey,I Don’t Wanna Cry Lyrics[Verse 1]\nOnce again ...,"[I, Don, t, Wanna, Cry, Lyrics, Verse, 1, Once...","[wanna, cry, sit, silence, said, done, emptine..."


In [38]:
!pip install pyLDAvis

In [39]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
vis_data = gensimvis.prepare(lda_model, gensim_corpus, dictionary)
#pyLDAvis.display(vis_data)
pyLDAvis.save_html(vis_data, './Lyrics_LDA_k_'+ str(num_topics) +'.html')

/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/navyajain/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/navyajain/opt/anac